* Repita o exercício 2 mas, em vez de parar a simulação depois de gerar N clientes, a simulação vai parar aplicando a regra do tamanho relativo do intervalo de confiança.


* Para isso, n vai crescer indefinidamente. Calcular   Wbarra   e o intervalo de confiança de 95%.

Se H/Wbarra <= 0,05, parar a simulação, anotar o valor de Wbarra, H e o valor de N.

* Caso contrário, a simulação continua. 

In [45]:
import random                      
import simpy # biblioteca de simulação
import math as mt
from tqdm import tqdm
import statistics as ss
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np

In [46]:
taxaEntrada = 9           # taxa de entrada: λ=9 clientes por segundo. 
taxaServico = 10          # taxa de serviço: μ=10 clientes por segundo.

In [47]:
def media(lista):
    soma = 0
    for i in range(len(lista)):
        soma += lista[i]
    return soma/len(lista)

def varAmostral(lista):
    m = media(lista)
    soma = 0
    for i in range(len(lista)):
        soma += (lista[i]-m)**2
    return soma/(len(lista)-1)
      
def desPadrao(lista):
    dp = mt.sqrt(varAmostral(lista))
    return dp

def exponencial(b):
    u = random.random()
    x = -b * mt.log(1-u)

    return(x)

In [48]:
def tempoQuantiMedia(quanti, media):
    return [-(1/media) *  np.log(1 - np.random.uniform(0,1)) for _ in range(quanti-1)]

In [49]:
def mm1(n, lamb, mi):
    medioEsperaFila = tempoQuantiMedia(n, lamb)
    servicos = tempoQuantiMedia(n, mi)
    filaEsperas = [0]
    espera = 0
    i = 0
    while i < len(medioEsperaFila):
        servicoCompleto = servicos[i] + filaEsperas[-1]
        if (medioEsperaFila[i] <= servicoCompleto):
             espera = (servicos[i] - medioEsperaFila[i]) + filaEsperas[-1]
        else:
             espera = 0
        filaEsperas.append(espera)
        i+=1
    return filaEsperas

In [50]:
def mser5y(nClientes, x):
    auxList = []
    aux = 0
    z = []
    i = j =0
    while i < nClientes:
        while j < 5:
            auxList.append(x[i])
            i +=1
            j +=1
        z.append(media(auxList))
        auxList.clear()
        j=0
    mser5yList = []
    k = nClientes/5
    meio = k/2
    d = 0
    while d < meio:
        desvpZ = ss.fmean(z)
        aux = desvpZ / mt.sqrt(k-d)
        mser5yList.append(aux)
        d += 1
        del z[0]
    menor = min(mser5yList)              
    posicao = mser5yList.index(menor)
    return posicao

In [51]:
def batchmeans (lista, b, m): #funcao: lista dos elementos no estado estacionário, b = blocos, m = tamanho dos blocos
    n = b*m         #total de observações
    i = 0
    j = 0
    listaux = []
    listaMedias = []
    while i < n:
        while j < m:
            listaux.append(lista[i])
            i += 1
            j += 1
        aux = ss.mean(listaux)
        listaMedias.append(aux)     #lista com as médias dos blocos
        j = 0
        listaux.clear()
    return(listaMedias)

In [52]:
def ranking (lista):    #cria lista de rankings 
    lranking = []
    i = 0
    l = 0
    soma = 0
    while l < len(lista):
        while i < len(lista):
            if lista[i] <= lista[l]:
                soma += 1
            i += 1
        lranking.append(soma)
        soma = 0
        i = 0
        l += 1
    return(lranking)


In [53]:
def estatisticaVN(lista):
    tf = (len(lista) - 1) #B-1
    i = 0
    somatorio1 = 0
    somatorio2 = 0

    while i < tf:
        sub = lista[i] - lista[i+1]
        somatorio1 += mt.pow(sub, 2)
        i += 1

    mediaR = ss.mean(lista)
    i = 0
    while i < len(lista): #B
        sub = lista[i] - mediaR
        somatorio2 += mt.pow(sub, 2)
        i += 1 
    rvn = somatorio1 / somatorio2
    return(rvn)

In [57]:
def sts (lista, b, m):
    n = b*m
    i = 0
    j = 0
    listaux = []
    listaA = []

    while i < n:
        while j < m:
            aux = lista[i]
            mult = ((m + 1)/2) - (j+1)
            listaux.append(mult*aux)
            i += 1
            j += 1
        
        soma = sum(listaux)
        listaA.append(soma) 
        j = 0
        listaux.clear()

    return(listaA)

In [58]:
def E(taxaEntrada, taxaServico):
    p = taxaEntrada/taxaServico
    e = p*((1/taxaServico)/(1-p))
    return e

In [59]:
def truncagem(nClientes):
    i =0
    b = 20
    m = 100
    n = nClientes
    x = mm1(n, 9, 10) # 10^5
    mser5yList =  mser5y(n, x)

    while mser5yList == 0:
        print('Ainda não se encontrou o ponto de truncagem.\n')
        print('A coletar mais informações até que se encontre...')
        n += 2000  
        x = mm1(n, 9, 10)
        mser5yList = mser5y(n, x)

    print('Encontrou o fim do transiente...')
    print('Eliminando as observações do estado transiente...')
    print('Tamanho de M/M/1 final:', len(x))
    pontoTruc = mser5yList * 5
    print(f'Encontrou-se o ponto de truncagem. Na posição {pontoTruc} da fila')
    #eliminando transiente
    while i < pontoTruc:
        del x[0]
        i +=1

    print('Tamanho de M/M/1 após o corte: ', len(x), '\nAgora está no estado estacionario.\n')
    n = b*m
    coletaobs = []
    i = 0
    while i < n:   
        coletaobs.append(x[i])
        i += 1
    batch = batchmeans(coletaobs, b, m)
    rank = ranking(batch)
    rvn = estatisticaVN(rank)
    
    valorWbarra = ss.mean(coletaobs)
    tstudent = list(st.t.interval(1-(0.01/2), b-1))
    t = tstudent[1]
    lsts = sts(coletaobs, b, m)
    soma = 0
    for l in lsts:
        soma += mt.pow(l, 2)
    vt = (12/(mt.pow(m, 3) - m)) * soma
    h = t * mt.sqrt(vt/n)
    
    while (h/valorWbarra) <= 0.05:
        m += 10
        temp = len(coletaobs)

        while len(coletaobs) < (b*m):
            coletaobs.append(x[temp])
            temp += 1

        lsts = sts(coletaobs, b, m)
        valorWbarra = ss.mean(coletaobs)
        tstudent = list(st.t.interval(1-(0.01/2), b-1))
        t = tstudent[1]

        soma = 0
        for l in lsts:
            soma += mt.pow(l, 2)

        vt = (12/(mt.pow(m, 3) - m)) * soma
        h = t * mt.sqrt(vt/n)
    return(valorWbarra, h, n)


In [61]:
v1, v2, v3 = truncagem(10000)

Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 16000
Encontrou-se o ponto de truncagem. Na posição 1945 da fila
Tamanho de M/M/1 após o corte:  14055 
Agora está no estado estacionario.



In [63]:
print(f' Wbarra = {v1} H = {v2} e o valor de N = {v3}.')

 Wbarra = 0.4813863145792686 H = 0.566681311530511 e o valor de N = 2000.
